***Code for Extracting the Stack of images from Video.***

***3.1 Image Generation***

In [ ]:
from moviepy.editor import VideoFileClip
import os
import imageio

In [ ]:
def video_to_images(video_path, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Load video file
    clip = VideoFileClip(video_path)

    # Iterate over each frame and save it as an image
    for i, frame in enumerate(clip.iter_frames()):
        # Construct the output image filename
        image_filename = os.path.join(output_folder, f"frame_{i:04d}.jpg")

        # Save the frame as an image using imageio
        imageio.imwrite(image_filename, frame)

    # Close the video clip
    clip.close()

In [ ]:
# Example usage
video_path = "C:\\Users\\harsh\\OneDrive\\Desktop\\3CYL_Flow.mp4"
output_folder = "output_images"
video_to_images(video_path, output_folder)

***Implementation of POD using PCA on Stacks of images.***

***3.2 Execute POD***

***3.3 Analyse POD Modes***

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D

In [ ]:
image_folder_path = '/content/drive/MyDrive/output_images'

In [ ]:
# Function to load images and convert them to grayscale
def load_images(image_folder_path, batch_size=50, resize=True, target_size=(400,400)):
    images = []
    for filename in os.listdir(image_folder_path):
        img_path = os.path.join(image_folder_path, filename)
        if img_path.endswith('.png') or img_path.endswith('.jpg'):
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)  # Read image in color mode
            if resize:
                img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            images.append(img_gray.flatten())
            if len(images) == batch_size:
                yield images
                images = []
    if images:
        yield images

In [ ]:
# Function to perform PCA
def perform_pca(image_generator, n_components=None):
    images = []
    for batch in image_generator:
        images.extend(batch)
    pca = PCA(n_components=n_components)
    pca.fit(images)
    return pca

In [ ]:
# Function to plot energy distribution
def plot_energy_distribution(pca):
    energy_percentage = pca.explained_variance_ratio_
    modes = np.arange(1, len(energy_percentage) + 1)
    plt.plot(modes, energy_percentage, marker='o')
    plt.xlabel('Mode')
    plt.ylabel('Energy Percentage')
    plt.title('Energy Distribution among Modes')
    plt.grid(True)
    plt.show()

In [ ]:
# Function to report top energy modes
def report_top_modes(pca):
    energy_percentage = pca.explained_variance_ratio_
    top_modes_indices = np.argsort(energy_percentage)[::-1][:10]
    top_modes_energy = energy_percentage[top_modes_indices]

    print("Top 10 Energy Modes:")
    for i, mode_idx in enumerate(top_modes_indices):
        print(f"Mode {i+1}: Energy = {top_modes_energy[i]*100:.2f}%")
    return top_modes_indices, top_modes_energy

In [ ]:
# Function to display top mode images
def display_top_mode_images(pca, top_modes_indices, target_size=(400, 400), image_size=(8, 8)):
    num_modes = len(top_modes_indices)
    num_cols = 5
    num_rows = (num_modes + num_cols - 1) // num_cols

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(image_size[0]*num_cols, image_size[1]*num_rows))
    fig.subplots_adjust(hspace=0.3, wspace=0.1)

    for i, mode_idx in enumerate(top_modes_indices):
        # Generate synthetic image using PCA component
        synthetic_coefficients = np.zeros(pca.n_components_)
        synthetic_coefficients[mode_idx] = 3 * np.sqrt(pca.explained_variance_[mode_idx])
        synthetic_image_flat = np.dot(synthetic_coefficients, pca.components_) + pca.mean_
        synthetic_image = np.reshape(synthetic_image_flat, target_size)

        # Convert to 8-bit unsigned integer
        synthetic_image = synthetic_image.astype(np.uint8)

        row = i // num_cols
        col = i % num_cols
        ax = axes[row, col]
        ax.imshow(synthetic_image, cmap=None)  # Set cmap to None
        ax.set_title(f"Mode {i+1}")
        ax.axis('off')

    # Hide empty subplots
    for i in range(num_modes, num_rows * num_cols):
        row = i // num_cols
        col = i % num_cols
        ax = axes[row, col]
        ax.axis('off')

    plt.show()


In [ ]:
# Function to load and preprocess images from the folder
def load_and_preprocess_images(folder_path, target_height=128, target_width=128):
    images = []
    for filename in os.listdir(folder_path):
        img = Image.open(os.path.join(folder_path, filename))
        # Resize image
        img = img.resize((target_width, target_height))
        # Convert image to numpy array and normalize
        img_array = np.array(img) / 255.0
        images.append(img_array)
    return np.array(images)

In [ ]:
batch_size = 50  # Adjust based on your memory constraints
image_generator = load_images(image_folder_path, batch_size=batch_size)

In [ ]:
# Perform PCA
pca = perform_pca(image_generator)

In [ ]:
# Load and preprocess images from the folder
image_stack = load_and_preprocess_images(image_folder_path)

In [ ]:
# Vectorize the data
num_images, height, width, channels = image_stack.shape
data = image_stack.reshape(num_images, -1)

In [ ]:
# Perform PCA
pca1 = PCA()
pca1.fit(data)

In [ ]:
# Plot the mean image
mean_image = pca1.mean_.reshape(height, width, channels)
plt.imshow(mean_image)
plt.title('Mean Image')
plt.axis('off')
plt.show()

In [ ]:
# Plot Energy Distribution
plot_energy_distribution(pca)

# Report Top 10 Energy Modes
top_modes_indices, _ = report_top_modes(pca)

In [ ]:
# Display Images corresponding to Top Modes
display_top_mode_images(pca, top_modes_indices)

***Code for implimentation of adding Noise with Magnitude 20%, 40%, 60% and 80%***

***4. Noise!***

***4.1 Adding Noise***

In [ ]:
# Define path to the folder containing original images
original_image_folder_path = '/content/drive/MyDrive/output_images'
# Define path to the folder to save noisy images
noisy_image_folder_path = '/content/drive/MyDrive/noisy_images'

Gaussian Noise.

In [ ]:
# Function to add Gaussian noise to an image
def add_gaussian_noise(image, magnitude):
    mean = 0
    std = magnitude * np.max(image)
    noisy_image = image + np.random.normal(mean, std, image.shape)
    return np.clip(noisy_image, 0, 1)

Salt and Pepper noise.

In [ ]:
# Function to add salt and pepper noise to an image
def add_salt_and_pepper_noise(image, magnitude):
    noisy_image = np.copy(image)
    salt_vs_pepper = 0.5
    salt = np.ceil(magnitude * np.max(image) * salt_vs_pepper)
    pepper = np.ceil(magnitude * np.max(image) * (1.0 - salt_vs_pepper))
    # Add salt noise
    coords = [np.random.randint(0, i - 1, int(salt)) for i in image.shape]
    noisy_image[coords] = 1
    # Add pepper noise
    coords = [np.random.randint(0, i - 1, int(pepper)) for i in image.shape]
    noisy_image[coords] = 0
    return noisy_image

Speckle_Noise.

In [ ]:
# Function to add speckle noise to an image
def add_speckle_noise(image, magnitude):
    noisy_image = image + magnitude * np.random.normal(0, 1, image.shape)
    return np.clip(noisy_image, 0, 1)

In [ ]:
# Define noise magnitudes and types
noise_magnitudes = [0.2, 0.4, 0.6, 0.8]  # Noise magnitudes as a percentage of maximum magnitude
noise_types = ['Gaussian', 'Salt_and_pepper', 'Speckle']

***Saving Noisy Images.***

In [ ]:
# Create directories for noisy images
for noise_type in noise_types:
    for magnitude in noise_magnitudes:
        folder_name = f'noisy_{noise_type}_magnitude_{int(magnitude * 100)}'
        folder_path = os.path.join(noisy_image_folder_path, folder_name)
        os.makedirs(folder_path, exist_ok=True)

In [ ]:
# Add noise to original images and save noisy images
for magnitude in noise_magnitudes:
    for i, image in enumerate(image_stack):
        for noise_type in noise_types:
            if noise_type == 'Gaussian':
                noisy_image = add_gaussian_noise(image, magnitude)
            elif noise_type == 'Salt_and_pepper':
                noisy_image = add_salt_and_pepper_noise(image, magnitude)
            elif noise_type == 'Speckle':
                noisy_image = add_speckle_noise(image, magnitude)
            # Save noisy image
            folder_name = f'noisy_{noise_type}_magnitude_{int(magnitude * 100)}'
            folder_path = os.path.join(noisy_image_folder_path, folder_name)
            noisy_image_name = f'noisy_{i}.png'
            noisy_image_path = os.path.join(folder_path, noisy_image_name)
            plt.imsave(noisy_image_path, noisy_image, cmap='gray')  # Save noisy image

***4.2 Effect on POD Modes***

In [ ]:
image_folder_path20 = '/content/drive/MyDrive/noisy_images/noisy_Gaussian_magnitude_20'

In [ ]:
batch_size = 50  # Adjust based on your memory constraints
image_generator = load_images(image_folder_path20, batch_size=batch_size)

In [ ]:
# Perform PCA
pca = perform_pca(image_generator)

In [ ]:
# Plot Energy Distribution
plot_energy_distribution(pca)

# Report Top 10 Energy Modes
top_modes_indices, _ = report_top_modes(pca)

In [ ]:
# Display Images corresponding to Top Modes
display_top_mode_images(pca, top_modes_indices)

In [ ]:
# Calculate cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative explained variance ratio
plt.plot(cumulative_variance_ratio, marker='o')
plt.xlabel('Number of Modes')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
image_folder_path40 = '/content/drive/MyDrive/noisy_images/noisy_Gaussian_magnitude_40'

In [ ]:
batch_size = 50  # Adjust based on your memory constraints
image_generator = load_images(image_folder_path40, batch_size=batch_size)

In [ ]:
# Perform PCA
pca = perform_pca(image_generator)

In [ ]:
# Plot Energy Distribution
plot_energy_distribution(pca)

# Report Top 10 Energy Modes
top_modes_indices, _ = report_top_modes(pca)

In [ ]:
# Calculate cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative explained variance ratio
plt.plot(cumulative_variance_ratio, marker='o')
plt.xlabel('Number of Modes')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
# Display Images corresponding to Top Modes
display_top_mode_images(pca, top_modes_indices)

In [ ]:
image_folder_path60 = '/content/drive/MyDrive/noisy_images/noisy_Gaussian_magnitude_60'

In [ ]:
batch_size = 50  # Adjust based on your memory constraints
image_generator = load_images(image_folder_path60, batch_size=batch_size)

In [ ]:
# Perform PCA
pca = perform_pca(image_generator)

In [ ]:
# Plot Energy Distribution
plot_energy_distribution(pca)

# Report Top 10 Energy Modes
top_modes_indices, _ = report_top_modes(pca)

In [ ]:
# Calculate cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative explained variance ratio
plt.plot(cumulative_variance_ratio, marker='o')
plt.xlabel('Number of Modes')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
# Display Images corresponding to Top Modes
display_top_mode_images(pca, top_modes_indices)

In [ ]:
image_folder_path80 = '/content/drive/MyDrive/noisy_images/noisy_Gaussian_magnitude_80'

In [ ]:
batch_size = 50  # Adjust based on your memory constraints
image_generator = load_images(image_folder_path80, batch_size=batch_size)

In [ ]:
# Perform PCA
pca = perform_pca(image_generator)

In [ ]:
# Plot Energy Distribution
plot_energy_distribution(pca)

# Report Top 10 Energy Modes
top_modes_indices, _ = report_top_modes(pca)

In [ ]:
# Calculate cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative explained variance ratio
plt.plot(cumulative_variance_ratio, marker='o')
plt.xlabel('Number of Modes')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
# Display Images corresponding to Top Modes
display_top_mode_images(pca, top_modes_indices)

In [ ]:
image_folder_path201 = '/content/drive/MyDrive/noisy_images/noisy_Salt_and_pepper_magnitude_20'

In [ ]:
batch_size = 50  # Adjust based on your memory constraints
image_generator = load_images(image_folder_path201, batch_size=batch_size)

In [ ]:
# Perform PCA
pca = perform_pca(image_generator)

In [ ]:
# Plot Energy Distribution
plot_energy_distribution(pca)

# Report Top 10 Energy Modes
top_modes_indices, _ = report_top_modes(pca)

In [ ]:
# Calculate cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative explained variance ratio
plt.plot(cumulative_variance_ratio, marker='o')
plt.xlabel('Number of Modes')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
# Display Images corresponding to Top Modes
display_top_mode_images(pca, top_modes_indices)

In [ ]:
image_folder_path401 = '/content/drive/MyDrive/noisy_images/noisy_Salt_and_pepper_magnitude_40'

In [ ]:
batch_size = 50  # Adjust based on your memory constraints
image_generator = load_images(image_folder_path401, batch_size=batch_size)

In [ ]:
# Perform PCA
pca = perform_pca(image_generator)

In [ ]:
# Plot Energy Distribution
plot_energy_distribution(pca)

# Report Top 10 Energy Modes
top_modes_indices, _ = report_top_modes(pca)

In [ ]:
# Calculate cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative explained variance ratio
plt.plot(cumulative_variance_ratio, marker='o')
plt.xlabel('Number of Modes')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
# Display Images corresponding to Top Modes
display_top_mode_images(pca, top_modes_indices)

In [ ]:
image_folder_path601 = '/content/drive/MyDrive/noisy_images/noisy_Salt_and_pepper_magnitude_60'

In [ ]:
batch_size = 50  # Adjust based on your memory constraints
image_generator = load_images(image_folder_path601, batch_size=batch_size)

In [ ]:
# Perform PCA
pca = perform_pca(image_generator)

In [ ]:
# Plot Energy Distribution
plot_energy_distribution(pca)

# Report Top 10 Energy Modes
top_modes_indices, _ = report_top_modes(pca)

In [ ]:
# Calculate cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative explained variance ratio
plt.plot(cumulative_variance_ratio, marker='o')
plt.xlabel('Number of Modes')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
# Display Images corresponding to Top Modes
display_top_mode_images(pca, top_modes_indices)

In [ ]:
image_folder_path801 = '/content/drive/MyDrive/noisy_images/noisy_Salt_and_pepper_magnitude_80'

In [ ]:
batch_size = 50  # Adjust based on your memory constraints
image_generator = load_images(image_folder_path801, batch_size=batch_size)

In [ ]:
# Perform PCA
pca = perform_pca(image_generator)

In [ ]:
# Plot Energy Distribution
plot_energy_distribution(pca)

# Report Top 10 Energy Modes
top_modes_indices, _ = report_top_modes(pca)

In [ ]:
# Calculate cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative explained variance ratio
plt.plot(cumulative_variance_ratio, marker='o')
plt.xlabel('Number of Modes')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
# Display Images corresponding to Top Modes
display_top_mode_images(pca, top_modes_indices)

In [ ]:
image_folder_path202 = '/content/drive/MyDrive/noisy_images/noisy_Speckle_magnitude_20'

In [ ]:
batch_size = 50  # Adjust based on your memory constraints
image_generator = load_images(image_folder_path202, batch_size=batch_size)

In [ ]:
# Perform PCA
pca = perform_pca(image_generator)

In [ ]:
# Plot Energy Distribution
plot_energy_distribution(pca)

# Report Top 10 Energy Modes
top_modes_indices, _ = report_top_modes(pca)

In [ ]:
# Calculate cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative explained variance ratio
plt.plot(cumulative_variance_ratio, marker='o')
plt.xlabel('Number of Modes')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
# Display Images corresponding to Top Modes
display_top_mode_images(pca, top_modes_indices)

In [ ]:
image_folder_path402 = '/content/drive/MyDrive/noisy_images/noisy_Speckle_magnitude_40'

In [ ]:
batch_size = 50  # Adjust based on your memory constraints
image_generator = load_images(image_folder_path402, batch_size=batch_size)

In [ ]:
# Perform PCA
pca = perform_pca(image_generator)

In [ ]:
# Plot Energy Distribution
plot_energy_distribution(pca)

# Report Top 10 Energy Modes
top_modes_indices, _ = report_top_modes(pca)

In [ ]:
# Calculate cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative explained variance ratio
plt.plot(cumulative_variance_ratio, marker='o')
plt.xlabel('Number of Modes')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
# Display Images corresponding to Top Modes
display_top_mode_images(pca, top_modes_indices)

In [ ]:
image_folder_path602 = '/content/drive/MyDrive/noisy_images/noisy_Speckle_magnitude_60'

In [ ]:
batch_size = 50  # Adjust based on your memory constraints
image_generator = load_images(image_folder_path602, batch_size=batch_size)

In [ ]:
# Perform PCA
pca = perform_pca(image_generator)

In [ ]:
# Plot Energy Distribution
plot_energy_distribution(pca)

# Report Top 10 Energy Modes
top_modes_indices, _ = report_top_modes(pca)

In [ ]:
# Calculate cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative explained variance ratio
plt.plot(cumulative_variance_ratio, marker='o')
plt.xlabel('Number of Modes')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
# Display Images corresponding to Top Modes
display_top_mode_images(pca, top_modes_indices)

In [ ]:
image_folder_path802 = '/content/drive/MyDrive/noisy_images/noisy_Speckle_magnitude_80'

In [ ]:
batch_size = 50  # Adjust based on your memory constraints
image_generator = load_images(image_folder_path802, batch_size=batch_size)

In [ ]:
# Perform PCA
pca = perform_pca(image_generator)

In [ ]:
# Plot Energy Distribution
plot_energy_distribution(pca)

# Report Top 10 Energy Modes
top_modes_indices, _ = report_top_modes(pca)

In [ ]:
# Calculate cumulative explained variance ratio
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Plot cumulative explained variance ratio
plt.plot(cumulative_variance_ratio, marker='o')
plt.xlabel('Number of Modes')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio')
plt.grid(True)
plt.show()

In [ ]:
# Display Images corresponding to Top Modes
display_top_mode_images(pca, top_modes_indices)

***5. Super-Resolving***

***Implementation of Denoising using CNN arcitecture on Noisy_Gaussian with magnitude 20%.***

In [ ]:
# Load and preprocess noisy and clean images
def load_images(image_folder_path, target_size=(256, 256)):
    images = []
    for filename in os.listdir(image_folder_path):
        img_path = os.path.join(image_folder_path, filename)
        if img_path.endswith('.png') or img_path.endswith('.jpg'):
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale mode
            img = cv2.resize(img, target_size, interpolation=cv2.INTER_AREA)
            img = img.astype('float32') / 255.0  # Normalize pixel values to [0, 1]
            images.append(img)
    return np.array(images)


In [ ]:
# Load noisy and clean images
noisy_image_folder_path = '/content/drive/MyDrive/noisy_images/noisy_Gaussian_magnitude_20'
clean_image_folder_path = '/content/drive/MyDrive/output_images'
noisy_images = load_images(noisy_image_folder_path)
clean_images = load_images(clean_image_folder_path)

In [ ]:
# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(noisy_images, clean_images, test_size=0.2, random_state=42)

In [ ]:
# Define the CNN architecture (simple U-Net-like model)
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(256, 256, 1)),
    MaxPooling2D((2, 2), padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(1, (3, 3), activation='sigmoid', padding='same')])

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
history = model.fit(X_train, y_train, batch_size=32, epochs=10, validation_data=(X_val, y_val))

In [ ]:
# Plot training history (loss)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Evaluate the model
loss = model.evaluate(X_val, y_val)
print("Validation Loss:", loss)

In [ ]:
# Plot example denoising results
num_examples = 5
plt.figure(figsize=(10, 4))
for i in range(num_examples):
    plt.subplot(2, num_examples, i+1)
    plt.imshow(X_val[i].reshape(256, 256), cmap=None)
    plt.title('Noisy Image')
    plt.axis('off')

    plt.subplot(2, num_examples, i+1+num_examples)
    denoised_image = model.predict(np.expand_dims(X_val[i], axis=0)).reshape(256, 256)
    plt.imshow(denoised_image, cmap=None)
    plt.title('Denoised Image')
    plt.axis('off')

plt.tight_layout()
plt.show()